In [ ]:
%%shell

wget https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
gdown https://drive.google.com/uc?id=1FEyZKG-Z61OUp8aHfc3r7TH0ef5XfyAN
tar -xzf flower_photos.tgz
tar -xzf flower_data.tar.gz

--2020-10-06 11:26:24--  https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 172.253.120.128, 74.125.133.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228813984 (218M) [application/x-compressed-tar]
Saving to: ‘flower_photos.tgz’

flower_photos.tgz   100%[===================>] 218.21M   137MB/s    in 1.6s    

2020-10-06 11:26:26 (137 MB/s) - ‘flower_photos.tgz’ saved [228813984/228813984]

Downloading...
From: https://drive.google.com/uc?id=1FEyZKG-Z61OUp8aHfc3r7TH0ef5XfyAN
To: /content/flower_data.tar.gz
100% 70.9k/70.9k [00:00<00:00, 60.4MB/s]


In [ ]:
classes = ['daisy','dandelion','roses','sunflowers','tulips']

In [ ]:
import os
import cv2
import time
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score,roc_auc_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

In [ ]:
SEED = 42
BATCH_SIZE = 32
SIZE = [224,224]
LR = 0.0001
WEIGHT_DECAY = 0
EPOCHS = 20
TTA = 4

In [ ]:
def seed_everything(SEED):
    np.random.seed(SEED);
    torch.manual_seed(SEED);
    torch.cuda.manual_seed(SEED)
    #torch.backends.cudnn.deterministic = False
    #torch.backends.cudnn.benchmark = False

seed_everything(SEED) 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("CURRENT DEVICE: {0}".format(device))
print("GPU NAME: {0}".format(torch.cuda.get_device_name(device.index))) if torch.cuda.is_available() else None

CURRENT DEVICE: cuda:0
GPU NAME: Tesla K80


In [ ]:
train_df = pd.read_csv('train_flowers.csv')
test_df  = pd.read_csv('test_flowers.csv')
valid_df = pd.read_csv('valid_flowers.csv')

In [ ]:
 class flower_dataset(Dataset):
    def __init__(self,df,transform=None,test=False):
        self.df = df
        self.transform = transform
        self.test = test
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image_path = self.df.iloc[idx]['image_path']
        image = plt.imread(os.path.join('flower_photos',image_path))

        if self.transform is not None:
          image = self.transform(image)

        if self.test==False:
            label = self.df.iloc[idx]['target']
            return {'image': image,'label': label }

        elif self.test==True:
            return {'image':image}

In [ ]:
transform = {
    'train' : transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(SIZE),
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomVerticalFlip(0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [ ]:
train_dataset = flower_dataset(df=train_df, transform=transform['train'])
train_dataloader= DataLoader(train_dataset , batch_size=BATCH_SIZE, num_workers=4, shuffle=True, drop_last=True)

valid_dataset = flower_dataset(df=valid_df, transform=transform['valid'])
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=False)

In [ ]:
def plot_transform(index,num_images=7):
    plt.figure(figsize=(30,10))
    for i in range(1,num_images+1):
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        plt.subplot(1,num_images+1,i);plt.axis('off')
        return_dict = train_dataset.__getitem__(index)
        image = return_dict['image'].numpy().transpose((1, 2, 0))
        image = std * image + mean
        image = np.clip(image, 0, 1)
        plt.imshow(image)
    print(classes[return_dict['label']])

#plot_transform(index=4)

In [ ]:
def get_model():
  model = models.resnet18(pretrained=True)
  model.fc = nn.Linear(in_features=model.fc.in_features, out_features=5, bias=True)
  model = model.to(device)
  return model

In [ ]:
flower_model = get_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(flower_model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=2, verbose=True, min_lr=0)

In [ ]:
results = pd.DataFrame(columns=['training_loss','training_accuracy','validation_loss','validation_accuracy'])
def train(model, criterion, optimizer,scheduler, train_dataloader, valid_dataloader):
    global results
    for epoch in range(EPOCHS):
        print('Epoch {}/{}'.format(epoch,EPOCHS-1))
        since = time.time()
        model.train()
        training_loss = []
        training_accuracy  = []
        for bi, data in enumerate(tqdm(train_dataloader, total=int(len(train_dataloader)))):
            inputs = data["image"].to(device, dtype=torch.float)
            labels = data["label"].to(device, dtype=torch.long)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                outputs = outputs.cpu().detach().numpy()
                labels = labels.cpu().numpy()
                training_accuracy.append(accuracy_score(np.argmax(outputs,axis=1),labels))
                training_loss.append(loss.item())
        print('Training accuracy: {:.4f} and Training Loss: {:.4f}'.format(np.mean(training_accuracy),np.mean(training_loss)))

        model.eval()
        validation_loss = []
        validation_accuracy = []
        with torch.no_grad():
            for bi,data in enumerate(tqdm(valid_dataloader,total=int(len(valid_dataloader)))):
                inputs = data["image"].to(device, dtype=torch.float)
                labels = data["label"].to(device, dtype=torch.long)
                outputs = model(inputs)
                loss = criterion(outputs,labels)
                outputs = outputs.cpu().detach().numpy()
                labels = labels.cpu().numpy()
                validation_accuracy.append(accuracy_score(np.argmax(outputs,axis=1),labels))
                validation_loss.append(loss.item())
        #torch.save(model.state_dict(), 'best_model.pt')
        #print("Saving model!")
        print('Validation accuracy: {:.4f} and Validation Loss: {:.4f}'.format(np.mean(validation_accuracy),np.mean(validation_loss)))
        res = pd.DataFrame([[np.mean(training_loss),np.mean(training_accuracy),np.mean(validation_loss),np.mean(validation_accuracy)]],columns=results.columns)
        results = pd.concat([results,res])
    return results.iloc[-1]

In [ ]:
train(flower_model, criterion, optimizer,scheduler, train_dataloader,valid_dataloader)

Epoch 0/19



Training accuracy: 0.9703 and Training Loss: 0.0962



Validation accuracy: 0.9337 and Validation Loss: 0.2209
Epoch 1/19



Training accuracy: 0.9840 and Training Loss: 0.0552



Validation accuracy: 0.9449 and Validation Loss: 0.2169
Epoch 2/19


...


Validation accuracy: 0.9173 and Validation Loss: 0.3611
Epoch 16/19



Training accuracy: 0.9939 and Training Loss: 0.0250



Validation accuracy: 0.9225 and Validation Loss: 0.3152
Epoch 17/19


Exception ignored in: <bound method tqdm.__del__ of 100%|<bar/>| 82/82 [06:04<00:00,  4.44s/it]>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/std.py", line 1061, in __del__
    def __del__(self):
KeyboardInterrupt: 


KeyboardInterrupt: ignored

In [ ]:
data = next(iter(train_dataloader))

In [ ]:
device = 'cpu'

In [ ]:
flower_model = get_model()

In [ ]:
data["image"].shape,data["label"].shape

(torch.Size([32, 3, 224, 224]), torch.Size([32]))

In [ ]:
inputs = data["image"].to(device, dtype=torch.float)
labels = data["label"].to(device, dtype=torch.long)

In [ ]:
flower_model.train()
optimizer.zero_grad()

In [ ]:
with torch.set_grad_enabled(True):
  outputs = flower_model(inputs)

In [ ]:
outputs.shape

torch.Size([32, 5])

In [ ]:
loss = criterion(outputs,labels)

In [ ]:
outputs = outputs.cpu().detach().numpy()

In [ ]:
labels = labels.cpu().numpy()

In [ ]:
accuracy_score(np.argmax(outputs,axis=1),labels)

0.21875